In [2]:
import sys
sys.path.insert(0, '../../build/gqcpy/')

import gqcpy
import numpy as np

In [8]:
def calculateOneOrbitalEntropy(D, d, site=0):
    """Calculate the one-orbital entropy, given the spin-resolved 1-particle DM and 2-particle DM."""

    # Construct the one-orbital DM. The formula is given in Boguslawski2015.
    i = site

    a = D.alpha.matrix()[i,i]
    b = D.beta.matrix()[i,i]
    c = d.alphaBeta().tensor()[i,i,i,i]

    rho = np.zeros((4,4))

    rho[0,0] = 1 - a - b + c
    rho[1,1] = a - c
    rho[2,2] = b - c
    rho[3,3] = c

    # To calculate the one-orbital entropy, we need the eigenvalues of the one-orbital RDM. But, since the one-orbital RDM is already diagonal, we can calculate the one-orbital entropy from those diagonal elements.
    S = 0  # The one-orbital entropy.
    for i in range(4):
        if not (abs(rho[i,i] < 1.0e-12)):  # rho[i,i] must be non-zero
            S -= rho[i,i] * np.log(rho[i,i])
    
    return S

In [4]:
A = gqcpy.AdjacencyMatrix.Cyclic(3)
t = 1.5
U = 1.0
H = gqcpy.HoppingMatrix(A, t, U)
hubbard_hamiltonian = gqcpy.HubbardHamiltonian(H)

In [5]:
onv_basis = gqcpy.SpinResolvedONVBasis(3, 2, 1)
solver = gqcpy.EigenproblemSolver.Dense()
environment = gqcpy.CIEnvironment.Dense(hubbard_hamiltonian, onv_basis)

expansion = gqcpy.CI(onv_basis).optimize(solver, environment).groundStateParameters()



In [6]:
D = expansion.calculateSpinResolved1DM()
d = expansion.calculateSpinResolved2DM()

In [9]:
S = calculateOneOrbitalEntropy(D, d, 0)
print(S)

TypeError: Unable to convert function return value to a Python type! The signature was
	(self: gqcpy.MixedSpinResolved2DMComponent_d) -> Eigen::Tensor<double, 4, 0, long>